In [ ]:
## using graphviz
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot
## 아래 있는것은 필수는 아닌데, 가끔 에러가 생길 때가 있어서, 그냥 같이 해줌.
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz
import pygraphviz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from gensim.models import Word2Vec
import networkx as nx
import pygraphviz as pgv
from random import shuffle

dot_filename = "/content/drive/MyDrive/0-cpg.dot"  # 실제 DOT 파일의 경로

# DOT 파일 파싱 및 그래프 생성 함수
def parse_dot_file(dot_filename):
    graph = pgv.AGraph(dot_filename)
    cpg_graph = nx.DiGraph()

    for edge in graph.edges():
        source = edge[0]
        target = edge[1]
        cpg_graph.add_edge(source, target)

    return cpg_graph

# DOT 파일 파싱 및 그래프 생성
cpg_graph = parse_dot_file(dot_filename)

# Betweenness Centrality 계산
betweenness_centrality = nx.betweenness_centrality(cpg_graph)

# 중요도가 큰 순서대로 노드 정렬
sorted_nodes = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)

# 랜덤 워크 생성 함수
def generate_random_walks(graph, num_walks, walk_length):
    walks = []
    for _ in range(num_walks):
        shuffle(sorted_nodes)  # 중요도가 큰 순서대로 섞음
        for node in sorted_nodes:
            walk = [node]
            current_node = node
            for _ in range(walk_length - 1):
                neighbors = list(graph.neighbors(current_node))
                if neighbors:
                    next_node = neighbors[0]
                    walk.append(next_node)
                    current_node = next_node
                else:
                    break
            walks.append(walk)
    return walks

# 랜덤 워크 생성
num_walks = 10
walk_length = 5
random_walks = generate_random_walks(cpg_graph, num_walks, walk_length)

# Word2Vec 모델 학습
model = Word2Vec(sentences=random_walks, vector_size=128, window=3, min_count=1, sg=1, workers=4, epochs=50)

# 중요한 노드에 대한 벡터 확인
most_important_node = sorted_nodes[0]
embedding_vector = model.wv[str(most_important_node)]
print("Most important node:", most_important_node)
print("Vector:", embedding_vector)